In [27]:
import numpy as np
import pandas as pd

import os
from scipy.integrate import trapezoid
import pathlib

In [28]:
base_dir = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2')

data_pt = os.path.join(base_dir, "data", "test_cases", "Trial_Full_System_90kW_3hr_Battery_with_Geothermal_Ramp_Limits","Trial_Full_System_90kW_3hr_Battery_with_Geothermal_Ramp_Limits_system_info.csv")
system_info = pd.read_csv(data_pt)

load_pt = os.path.join(base_dir, "data", "PySam_Outputs", "baseline_system_output.csv")
load = pd.read_csv(load_pt, index_col="Datetime", usecols=["Datetime", "Load (MW)"])
load.set_index(pd.DatetimeIndex(load.index), inplace=True)

In [29]:
x = np.arange(0, 8760, 5/60)
y = np.ravel(load.values)
annual_load = trapezoid(y, x) # MWh
annual_load_kwh = annual_load*1000

In [5]:
annual_load

np.float64(407720.13880330476)

In [108]:
ilr = 1.33
m2_per_wdc = 0.2*0.001
0.29 + 0.04/ilr + 0.16 + 0.18 + m2_per_wdc*(27.07 + 92.11) + 0.044 + 0.02 + 0.03 + m2_per_wdc*2.5 + 0.002

0.7804111879699247

best system

In [19]:
transmission_cost_per_mile = 6000000

geothermal_rating = 87000
geo_binary= 1

geothermal_capex = 775245440
hybrid_capex = 93674384  + 603005376

wind_op_costs = 30.85*system_info["Wind System Size"]

geothermal_op_costs = 118.41*geothermal_rating*geo_binary
geo_costs = {'Operating Costs': geothermal_op_costs, 'Capex': geothermal_capex}

pv_op_costs = 48*system_info["PV System Size"] + 180000

# pv_capex + battery_capex
hybrid_op_costs = pv_op_costs + wind_op_costs
hybrid_costs = {'Operating Costs': hybrid_op_costs, 'Capex': hybrid_capex}

system_costs = {}
annual_op_costs =  geothermal_op_costs + hybrid_op_costs
system_costs['Annual Operating Costs'] = annual_op_costs

inflation_rate = 1.0025
annual_op_costs_by_year = [annual_op_costs*inflation_rate**i for i in range(1, 26)]
total_op_costs = sum(annual_op_costs_by_year)
avg_annual_op_costs = total_op_costs/25
system_costs['Average Annual Operating Costs'] = avg_annual_op_costs

upfront_cost = geothermal_capex + hybrid_capex
system_costs['System Capex'] = upfront_cost
avg_annual_upfront_payment = upfront_cost/25
system_costs['Average Annual Capex Payments'] = avg_annual_upfront_payment

avg_annual_total_payment = avg_annual_upfront_payment + avg_annual_op_costs
system_costs['Required Annual Average Revenue'] = avg_annual_total_payment

avg_energy_rate = avg_annual_total_payment/annual_load_kwh
system_costs['Energy Rate (kWh)'] = avg_energy_rate


In [20]:
system_costs

{'Annual Operating Costs': 0    2.340343e+07
 dtype: float64,
 'Average Annual Operating Costs': 0    2.417947e+07
 dtype: float64,
 'System Capex': 1471925200,
 'Average Annual Capex Payments': 58877008.0,
 'Required Annual Average Revenue': 0    8.305648e+07
 dtype: float64,
 'Energy Rate (kWh)': 0    0.20371
 dtype: float64}

Remove wind case

In [25]:
transmission_cost_per_mile = 6000000

pv_rating = 320000
geothermal_rating = 87000
geo_binary= 1

wind_capex = 0
wind_op_costs = 0
# wind_op_costs = system_info["Wind Operating Cost"]
wind_costs = {'Operating Costs': wind_op_costs, 'Capex': wind_capex}

geothermal_capex = 775245440
geothermal_op_costs = 118.41*geothermal_rating*geo_binary
geo_costs = {'Operating Costs': geothermal_op_costs, 'Capex': geothermal_capex}

pv_capex = 771027840
# pv_op_costs = system_info["PV Operating Cost"]
pv_op_costs = 48*pv_rating + 180000

hybrid_capex = pv_capex
# pv_capex + battery_capex
hybrid_op_costs = pv_op_costs
hybrid_costs = {'Operating Costs': hybrid_op_costs, 'Capex': hybrid_capex}

system_costs = {}
annual_op_costs =  wind_op_costs + geothermal_op_costs + hybrid_op_costs
system_costs['Annual Operating Costs'] = annual_op_costs

inflation_rate = 1.0025
annual_op_costs_by_year = [annual_op_costs*inflation_rate**i for i in range(1, 26)]
total_op_costs = sum(annual_op_costs_by_year)
avg_annual_op_costs = total_op_costs/25
system_costs['Average Annual Operating Costs'] = avg_annual_op_costs

upfront_cost = geothermal_capex + hybrid_capex
real_interest_rate = 1 + (0.00614 - 0.0025)
real_upfront_cost = upfront_cost*real_interest_rate**25 
avg_annual_upfront_payment = real_upfront_cost/25
system_costs['System Capex'] = real_upfront_cost
system_costs['Average Annual Capex Payments'] = avg_annual_upfront_payment

avg_annual_total_payment = avg_annual_upfront_payment + avg_annual_op_costs
system_costs['Required Annual Average Revenue'] = avg_annual_total_payment

avg_energy_rate = avg_annual_total_payment/annual_load_kwh
system_costs['Energy Rate (kWh)'] = avg_energy_rate


In [26]:
system_costs

{'Annual Operating Costs': 25841670.0,
 'Average Annual Operating Costs': 26698565.498050522,
 'System Capex': 1693305407.5836902,
 'Average Annual Capex Payments': 67732216.3033476,
 'Required Annual Average Revenue': 94430781.80139813,
 'Energy Rate (kWh)': np.float64(0.23160686170313038)}

baseline

In [ ]:
transmission_cost_per_mile = 6000000

pv_rating = 320000
geothermal_rating = 87000
geo_binary= 0

wind_capex = system_info['Wind Cost']
wind_op_costs = 30.85*system_info["Wind System Size"]
# wind_op_costs = system_info["Wind Operating Cost"]
wind_costs = {'Operating Costs': wind_op_costs, 'Capex': wind_capex}

geothermal_cost_per_kwac = 6153.66
geothermal_capex = 26.6*transmission_cost_per_mile*geo_binary + geothermal_cost_per_kwac*87*1000
geothermal_op_costs = 118.41*geothermal_rating*geo_binary
geo_costs = {'Operating Costs': geothermal_op_costs, 'Capex': geothermal_capex}

pv_op_costs = 48*pv_rating + 180000

hybrid_capex = 38814670848
hybrid_op_costs = pv_op_costs
hybrid_costs = {'Operating Costs': hybrid_op_costs, 'Capex': hybrid_capex}

system_costs = {}
annual_op_costs =  wind_op_costs + geothermal_op_costs + hybrid_op_costs
system_costs['Annual Operating Costs'] = annual_op_costs

inflation_rate = 1.0025
annual_op_costs_by_year = [annual_op_costs*inflation_rate**i for i in range(1, 26)]
total_op_costs = sum(annual_op_costs_by_year)
avg_annual_op_costs = total_op_costs/25
system_costs['Average Annual Operating Costs'] = avg_annual_op_costs

upfront_cost = geothermal_capex + hybrid_capex
real_interest_rate = 1 + (0.00614 - 0.0025)
real_upfront_cost = upfront_cost*real_interest_rate**25 
avg_annual_upfront_payment = real_upfront_cost/25
system_costs['System Capex'] = real_upfront_cost
system_costs['Average Annual Capex Payments'] = avg_annual_upfront_payment

avg_annual_total_payment = avg_annual_upfront_payment + avg_annual_op_costs
system_costs['Required Annual Average Revenue'] = avg_annual_total_payment

avg_energy_rate = avg_annual_total_payment/annual_load_kwh
system_costs['Energy Rate (kWh)'] = avg_energy_rate


In [114]:
system_costs

{'Annual Operating Costs': 0    17421850.0
 Name: Wind System Size, dtype: float64,
 'Average Annual Operating Costs': 0    1.799955e+07
 Name: Wind System Size, dtype: float64,
 'System Capex': 43091758192.40371,
 'Average Annual Capex Payments': 1723670327.6961484,
 'Required Annual Average Revenue': 0    1.741670e+09
 Name: Wind System Size, dtype: float64,
 'Energy Rate (kWh)': 0    4.271729
 Name: Wind System Size, dtype: float64}